# Custom Dynamic Event Example

In [1]:
#r "nuget: Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13"
// #r "./TraceEvent.dll" <- Here is how to add your custom TraceEvent.dll

#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: Microsoft.Data.Analysis"
#r "nuget: Newtonsoft.Json"

using Etlx = Microsoft.Diagnostics.Tracing.Etlx;
using Microsoft.Data.Analysis;
using Microsoft.Diagnostics.Tracing.Analysis.GC;
using Microsoft.Diagnostics.Tracing.Analysis;
using Microsoft.Diagnostics.Tracing.Parsers.Clr;
using Microsoft.Diagnostics.Tracing;
using XPlot.Plotly;

using System.IO;
using Newtonsoft.Json;

Installed Packages Microsoft.Data.Analysis, 0.22.0 Microsoft.Diagnostics.Tracing.TraceEvent, 3.1.13 Newtonsoft.Json, 13.0.3 XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

Loading extensions from `Q:\.tools\.nuget\packages\microsoft.data.analysis\0.22.0\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `Q:\.tools\.nuget\packages\xplot.plotly.interactive\4.0.7\lib\net7.0\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

In [2]:
dotnet build -c Release "..\..\GC.Analysis.API" # /p:CustomTraceEvent=true and Optionally add: /p:PerfViewPath=C:\\Dev\\PerfView

  Determining projects to restore...
  All projects are up-to-date for restore.
  GC.Analysis.API -> C:\Users\musharm\source\repos\performance_oom\artifacts\bin\GC.Analysis.API\Release\net8.0\GC.Analysis.API.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:01.18


In [3]:
#r "..\..\..\..\..\..\artifacts\bin\GC.Analysis.API\Release\net8.0\GC.Analysis.API.dll"

using GC.Analysis.API;
using GC.Analysis.API.DynamicEvents;

# Dynamic Event

In [4]:
Analyzer analyzer = new Analyzer(@"C:\Users\musharm\source\repos\performance\artifacts\Results\2gb\2gb.run.0.etlx");
GCProcessData gcData = analyzer.GetProcessGCData("corerun").First();

In [5]:
public static DynamicEventSchema OOMDetailsSchema = new DynamicEventSchema
{
    DynamicEventName = "OOMDetails",
    Fields = new List<KeyValuePair<string, Type>>
    {
        KeyValuePair.Create("version", typeof(ushort)),
        KeyValuePair.Create("GCIndex", typeof(uint)),
        KeyValuePair.Create("Allocated", typeof(ushort)),
        KeyValuePair.Create("Reserved", typeof(ushort)),
        KeyValuePair.Create("AllocSize", typeof(uint)),
        KeyValuePair.Create("Reason", typeof(ushort)),
        KeyValuePair.Create("FGM", typeof(ushort)),
        KeyValuePair.Create("Size", typeof(uint)),
        KeyValuePair.Create("AvailablePageFileMB", typeof(uint)),
    },
    MaxOccurrence = 1000
};

DynamicEventSchema.Add(OOMDetailsSchema);

In [9]:
gcData.GCs.Select(gc => gc.DynamicEvents().OOMDetails) 

Error: System.Exception: Event OOMDetails does not have matching size
   at GC.Analysis.API.DynamicEvents.DynamicEventObject..ctor(GCDynamicEvent dynamicEvent, CompiledSchema schema) in C:\Users\musharm\source\repos\performance_oom\src\benchmarks\gc\GC.Infrastructure\GC.Analysis.API\DynamicEvents\DynamicEvents.cs:line 218
   at GC.Analysis.API.DynamicEvents.DynamicIndex..ctor(List`1 dynamicEvents) in C:\Users\musharm\source\repos\performance_oom\src\benchmarks\gc\GC.Infrastructure\GC.Analysis.API\DynamicEvents\DynamicEvents.cs:line 194
   at GC.Analysis.API.DynamicEvents.TraceGCExtensions.DynamicEvents(TraceGC traceGC) in C:\Users\musharm\source\repos\performance_oom\src\benchmarks\gc\GC.Infrastructure\GC.Analysis.API\DynamicEvents\DynamicEvents.cs:line 20
   at Submission#10.<>c.<<Initialize>>b__0_0(TraceGC gc)
   at GC.Analysis.API.GoodLinq.Select[T,R](IEnumerable`1 data, Func`2 map) in C:\Users\musharm\source\repos\performance_oom\src\benchmarks\gc\GC.Infrastructure\GC.Analysis

In [7]:
Console.WriteLine($"Elapsed Time between GCs for GC#2: {gcData.GCs[1].DynamicEvents().SizeAdaptationSample.ElapsedTimeBetweenGCs}");
Console.WriteLine($"TotalSOHStableSize for GC#4: {gcData.GCs[3].DynamicEvents().SizeAdaptationTuning.TotalSOHStableSize}");

Error: Microsoft.CSharp.RuntimeBinder.RuntimeBinderException: Cannot perform runtime binding on a null reference
   at CallSite.Target(Closure, CallSite, Object)
   at System.Dynamic.UpdateDelegates.UpdateAndExecute1[T0,TRet](CallSite site, T0 arg0)
   at Submission#8.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)